In [1]:
import numpy as np
import pandas as pd
import re
import warnings

warnings.filterwarnings('ignore')

In [2]:
DF = pd.read_csv('preprocessing_flats.csv', index_col = 'Unnamed: 0')

In [4]:
DF.count()

Cсылка                     1113365
Цена                       1113378
Время загрузки             1113378
Ближайшее метро            1071948
Адрес                      1111787
Этаж                       1113148
Площадь кухни               939012
Расстояние до метро        1113378
Количество комнат          1108678
Жилая площадь               893688
Вид объекта                1113378
Площадь                    1113149
Город                      1113378
Регион                     1113378
ID                         1113378
Тип объявления             1113378
Этажей в доме              1113129
lat                        1113374
lng                        1113374
Тип дома                    296979
Год постройки               217239
Пассажирский лифт            85841
Грузовой лифт                60788
Запланирован снос          1113378
Санузел                     320445
Ремонт                      610802
Статус                      148742
Высота потолков             195845
Вид из окон         

In [5]:
def prepr_district(string):
    string = string.lower()
    for item, value in [('район', ''), ('поселение', 'сп'), ('ё', 'е'), 
                        ('ново-переделкино', 'новопеределкино'), ('сп московский', 'гп московский')]:
        string = string.replace(item, value)
    return string.strip()


geo_df = pd.read_excel('Станции Московского метрополитена.xlsx')[['Station', 'Line', 'AdmArea', 'District']]
geo_df = geo_df.rename(columns = dict(zip(geo_df.columns, ['Метро', 'Ветка', 'Округ', 'Район'])))

geo_df['Метро'] = geo_df['Метро'].map(lambda x: x.lower().replace('ё', 'е'))
geo_df['Район'] = geo_df['Район'].map(prepr_district)

new_DF = DF.merge(geo_df.drop_duplicates(subset = 'Метро'), how='left', 
              left_on='Ближайшее метро', right_on='Метро').drop(columns = ['Метро'])

eco_df = pd.read_csv('eco_data.csv', index_col = 'Unnamed: 0')
eco_df['district'] = eco_df['district'].map(lambda x: x.lower())

new_DF = new_DF.merge(eco_df, left_on = 'Район', right_on = 'district', how = 'left').drop(columns = ['district'])

In [6]:
new_DF.count()

Cсылка                                      1113365
Цена                                        1113378
Время загрузки                              1113378
Ближайшее метро                             1071948
Адрес                                       1111787
                                             ...   
Район                                       1012135
Плотность населения                         1012135
Загруженность шоссе                         1012135
Площадь зеленых насаждений                  1012135
Влияние объектов негативного воздействия    1012135
Length: 62, dtype: int64

In [52]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer

def get_features(X_train, X_fill, target):

    cols_to_train = ['Цена', 'Этаж', 'Площадь кухни', 'Расстояние до метро', 'Количество комнат',
    'Жилая площадь', 'Тип дома', 'Площадь', 'Этажей в доме', 'lat', 'lng','Пассажирский лифт', 'Грузовой лифт', 
    'Год постройки', 'Запланирован снос', 'Санузел', 'Ремонт', 'Статус', 'Планировка', 'Ветка', 'Округ',
    'Плотность населения', 'Загруженность шоссе', 'Площадь зеленых насаждений']
    if target in cols_to_train:
        cols_to_train.remove(target)
    
    X_train, X_fill = X_train[cols_to_train], X_fill[cols_to_train]
    
    if 'Количество комнат' in X_train.columns:
        X_train['Количество комнат'], X_fill['Количество комнат'] = [df['Количество комнат'].map(lambda x: 
        re.sub('[c, с]тудия', '0.5', x.replace('> 9', '10')) if type(x) == str else x).astype(float) for df in [X_train, X_fill]]
        X_train['Количество комнат'], X_fill['Количество комнат'] = [df['Количество комнат'].where(df['Количество комнат'] < 10, 10)
                                                                                        for df in [X_train, X_fill]]
    ordinary_cols = []
    for col in X_train.columns:
        if type(X_train[col].loc[X_train[col].first_valid_index()]) == str:
            
            train_mode = X_train[col].mode()
            fill_mode = X_fill[col].mode()
            
            if len(train_mode) == 0:
                train_mode = fill_mode
            elif len(X_fill[col].mode()) == 0:
                fill_mode = train_mode
                
            X_train.loc[:, col] = X_train[col].fillna(train_mode[0])
            X_fill.loc[:, col] = X_fill[col].fillna(fill_mode[0])
            ordinary_cols.append(col)
            
            
        elif len(np.concatenate(tuple([X[col].dropna().unique() for X in [X_train, X_fill]]))) == 2:
            X_train.loc[:, col] = X_train[col].fillna(X_train[col].mode())
            X_fill.loc[:, col] = X_fill[col].fillna(X_fill[col].mode())
            
        else:
            X_train[col] = X_train[col].fillna(X_train[col].mean())
            X_fill[col] = X_fill[col].fillna(X_fill[col].mean())
            
    CT = make_column_transformer(
    (OneHotEncoder(drop = 'if_binary', handle_unknown = 'ignore'), ordinary_cols),
        remainder='passthrough') 
    CT.fit(X_train)
    CT.transform(X_fill)
    X_train = CT.transform(X_train)
    X_fill = CT.transform(X_fill)
    return X_train, X_fill
    
    
def fill_na_values(df, target):
    X_train = df[~df[target].isna()]
    X_fill = df[df[target].isna()]
    
    X_train, y_train = X_train[X_train.columns[X_train.columns != target]], X_train[target]
    X_fill, y_fill_index = X_fill[X_fill.columns[X_fill.columns != target]], X_fill.index
    
    X_train, X_fill = get_features(X_train, X_fill, target) 
    
    regr = True
    if type(y_train.loc[y_train.first_valid_index()]) == str or len(y_train.unique()) <= 2:
        model = RandomForestClassifier(n_estimators=30)
        labelencoder = LabelEncoder()
        labelencoder.fit(y_train)
        y_train = labelencoder.transform(y_train)
        regr = False
    else:
        model = RandomForestRegressor(n_estimators=30)
    print(f'Заполняемый признак: {target}')
    display(model)
    model.fit(X_train, y_train)
    print(f'Качество на обучении: {model.score(X_train, y_train)}', 
          end = '\n========================================\n\n')
    
    if regr:
        y_fill = model.predict(X_fill)
    else:
        y_fill = labelencoder.inverse_transform(model.predict(X_fill))
        
    y_fill = pd.Series(index = y_fill_index, data = y_fill)
    
    return df.fillna({target: y_fill})

In [53]:
new_DF.columns

Index(['Cсылка', 'Цена', 'Время загрузки', 'Ближайшее метро', 'Адрес', 'Этаж',
       'Площадь кухни', 'Расстояние до метро', 'Количество комнат',
       'Жилая площадь', 'Вид объекта', 'Площадь', 'Город', 'Регион', 'ID',
       'Тип объявления', 'Этажей в доме', 'lat', 'lng', 'Тип дома',
       'Год постройки', 'Пассажирский лифт', 'Грузовой лифт',
       'Запланирован снос', 'Санузел', 'Ремонт', 'Статус', 'Высота потолков',
       'Вид из окон', 'Способ продажи', 'Балкон или лоджия', 'Вид сделки',
       'Тёплый пол', 'Спортивная площадка', 'Детская площадка',
       'Закрытая территория', 'Спальные места', 'Кухня', 'Хранение одежды',
       'Посудомоечная машина', 'Кондиционер', 'Утюг', 'Водонагреватель', 'Фен',
       'Холодильник', 'Телевизор', 'Стиральная машина', 'Газ', 'Консьерж',
       'Мусоропровод', 'Подземная', 'За шлагбаумом во дворе',
       'Открытая во дворе', 'Наземная многоуровневая', 'Планировка', 'Ветка',
       'Округ', 'Район', 'Плотность населения', 'Загруженнос

In [55]:
filled_DF = new_DF.dropna(subset = 'Район')
for col in ['Площадь кухни', 'Жилая площадь', 'Тип дома', 'Вид из окон', 'Высота потолков', 'Статус',
            'Ремонт', 'Год постройки', 'Санузел', 'Консьерж', 'Балкон или лоджия', 'Пассажирский лифт', 'Количество комнат']:
    filled_DF = fill_na_values(filled_DF, col)

Заполняемый признак: Площадь кухни


RandomForestRegressor(n_estimators=30)

Качество на обучении: 0.9659005543744535

Заполняемый признак: Жилая площадь


RandomForestRegressor(n_estimators=30)

Качество на обучении: 0.9733950599972936

Заполняемый признак: Тип дома


...

Качество на обучении: 0.9972061274704341

Заполняемый признак: Пассажирский лифт


RandomForestClassifier(n_estimators=30)

Качество на обучении: 0.9983325335892514

Заполняемый признак: Количество комнат


RandomForestClassifier(n_estimators=30)

Качество на обучении: 0.9992986765123094



In [44]:
filled_DF.count()[:40]

Cсылка                  1012122
Цена                    1012135
Время загрузки          1012135
Ближайшее метро         1012135
Адрес                   1010724
Этаж                    1012135
Площадь кухни           1012135
Расстояние до метро     1012135
Количество комнат       1012135
Жилая площадь           1012135
Вид объекта             1012135
Площадь                 1012135
Город                   1012135
Регион                  1012135
ID                      1012135
Тип объявления          1012135
Этажей в доме           1012129
lat                     1012133
lng                     1012133
Тип дома                1012135
Год постройки           1012135
Пассажирский лифт       1012135
Грузовой лифт             58724
Запланирован снос       1012135
Санузел                 1012135
Ремонт                  1012135
Статус                   145282
Высота потолков         1012135
Вид из окон             1012135
Способ продажи           144762
Балкон или лоджия       1012135
Вид сдел

In [56]:
filled_DF.to_csv('final_flats_data.csv')